In [2]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import apimoex
import sqlite3

from utils.data_loader import index_history
from utils.data_loader import ticker_prices, get_ticker_history
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
from optimizer import optimizer_for_tickers

import warnings
warnings.filterwarnings('ignore')

In [4]:
from api.combined_opt_hist import run_optimizer
from core.history import get_portfolio_history

tickers = ['SBER', 'LKOH', 'ROSN']

result = optimizer_for_tickers(tickers)
weights = result.get("weights", {})
df_history, message = get_portfolio_history(weights, start_date='2022-01-01', end_date='2025-04-07')
result["history"] = df_history.to_dict(orient="records")
result["history_message"] = message
result

Expected annual return: 12.0%
Annual volatility: 35.0%
Sharpe Ratio: 0.34


{'weights': {'LKOH': 1.0, 'ROSN': 0.0, 'SBER': 0.0},
 'performance': {'return': np.float64(0.11963316448555236),
  'volatility': np.float64(0.3498837991702253),
  'sharpe_ratio': np.float64(0.34192256048799935)},
 'history': [{'tradedate': Timestamp('2022-01-03 00:00:00'),
   'portfolio_value': 6683.0},
  {'tradedate': Timestamp('2022-01-04 00:00:00'), 'portfolio_value': 6728.0},
  {'tradedate': Timestamp('2022-01-05 00:00:00'), 'portfolio_value': 6522.0},
  {'tradedate': Timestamp('2022-01-06 00:00:00'), 'portfolio_value': 6699.0},
  {'tradedate': Timestamp('2022-01-10 00:00:00'), 'portfolio_value': 6775.0},
  {'tradedate': Timestamp('2022-01-11 00:00:00'), 'portfolio_value': 6816.0},
  {'tradedate': Timestamp('2022-01-12 00:00:00'), 'portfolio_value': 6960.0},
  {'tradedate': Timestamp('2022-01-13 00:00:00'), 'portfolio_value': 6665.0},
  {'tradedate': Timestamp('2022-01-14 00:00:00'), 'portfolio_value': 6659.0},
  {'tradedate': Timestamp('2022-01-17 00:00:00'), 'portfolio_value': 65

In [16]:
df = pd.read_csv(
    r"C:\Users\sofya\Desktop\Export_ru_securities-list_20250405.csv",
    encoding='utf-8',  # кодировка Windows, часто используется в России
    sep=';',            # разделитель — точка с запятой
    on_bad_lines='skip' # игнорируем строки с ошибками
)
df_stocks = df[df['SUPERTYPE'] == 'Депозитарные расписки']
tickers = df_stocks['TRADE_CODE']
tickers_list = tickers.tolist()
tickers_list

['ETLN', 'FIXP', 'QIWI', 'OKEY', 'OZON', 'AGRO']

In [15]:
import os
import pandas as pd
import requests
import sqlite3
from time import sleep

DB_NAME = "moex_candles.db"
DATA_DIR = "csv_data"

os.makedirs(DATA_DIR, exist_ok=True)

# --- Создание таблиц SQLite ---
def create_tables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS securities (
            ticker TEXT PRIMARY KEY,
            name TEXT,
            boardid TEXT,
            type TEXT,
            currency TEXT,
            is_active INTEGER DEFAULT 1
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS candles (
            ticker TEXT,
            tradedate TEXT,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL,
            value REAL,
            boardid TEXT,
            PRIMARY KEY (ticker, tradedate)
        )
    """)

    conn.commit()
    conn.close()

# --- Получение исторических данных с MOEX ---
def fetch_candles(ticker, board='TQBR'):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/{board}/securities/{ticker}/candles.json"
    params = {
        "from": "2000-01-01",
        "interval": 24,
        "iss.meta": "off",
        "iss.json": "extended",
        "candles.columns": "begin,open,high,low,close,value,volume"
    }

    all_data = []
    start = 0

    while True:
        try:
            r = requests.get(url, params={**params, "start": start}, timeout=15)
            r.raise_for_status()
            data = r.json()

            if len(data) < 2 or "candles" not in data[1]:
                break

            candles = data[1]["candles"]
            if not candles:
                break

            all_data.extend(candles)

            start += 500
            sleep(0.2)  # чтобы не получить бан

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            break

    df = pd.DataFrame(all_data, columns=["begin", "open", "high", "low", "close", "value", "volume"])
    df.rename(columns={"begin": "tradedate"}, inplace=True)
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.date  # удаляем время
    df.insert(0, "ticker", ticker)
    df["boardid"] = board

    return df

# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Save to CSV
    df.to_csv(os.path.join(DATA_DIR, f"{ticker}.csv"), index=False)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()

# --- Пример работы ---
if __name__ == "__main__":
    create_tables()
    tickers = tickers_list  # заменишь на свой список

    for ticker in tickers:
        df = fetch_candles(ticker)
        if not df.empty:
            save_data(df)
            print(f"Saved: {ticker} ({len(df)} rows)")
        else:
            print(f"No data for {ticker}")


Saved: CBOM (2461 rows)
Saved: ALRS (3144 rows)
Saved: VTBR (3040 rows)
Saved: MDMG (1085 rows)
Saved: GEMC (899 rows)
Saved: VKCO (803 rows)
Saved: LENT (820 rows)
Saved: RUAL (2520 rows)
Saved: T (1344 rows)
Saved: HEAD (141 rows)
Saved: CNRU (2 rows)
Saved: ENPG (1288 rows)
Saved: YDEX (187 rows)
Saved: BSPB (2723 rows)
Saved: AQUA (2360 rows)
Saved: AFKS (2725 rows)
Saved: AFLT (4297 rows)
Saved: VSEH (193 rows)
Saved: GAZP (2725 rows)
Saved: GMKN (3042 rows)
Saved: RAGR (35 rows)
Saved: LSRG (3550 rows)
Saved: POSI (824 rows)
Saved: RENI (860 rows)
Saved: EUTR (353 rows)
Saved: IRAO (3386 rows)
Saved: X5 (69 rows)
Saved: LEAS (268 rows)
Saved: MVID (2719 rows)
Saved: MBNK (241 rows)
Saved: MAGN (2725 rows)
Saved: MTLR (4032 rows)
Saved: MTLRP (3440 rows)
Saved: MTSS (3625 rows)
Saved: MOEX (3020 rows)
Saved: LKOH (5415 rows)
Saved: BELU (1883 rows)
Saved: NLMK (2725 rows)
Saved: PIKK (3418 rows)
Saved: PLZL (2722 rows)
Saved: RTKM (5375 rows)
Saved: RTKMP (5369 rows)
Saved: SBER (

In [17]:
import os
import pandas as pd
import requests
import sqlite3
from time import sleep

DB_NAME = "moex_candles.db"
DATA_DIR = "csv_data"

os.makedirs(DATA_DIR, exist_ok=True)

# --- Создание таблиц SQLite ---
def create_tables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS securities (
            ticker TEXT PRIMARY KEY,
            name TEXT,
            boardid TEXT,
            type TEXT,
            currency TEXT,
            is_active INTEGER DEFAULT 1
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS candles (
            ticker TEXT,
            tradedate TEXT,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL,
            value REAL,
            boardid TEXT,
            PRIMARY KEY (ticker, tradedate)
        )
    """)

    conn.commit()
    conn.close()

# --- Получение исторических данных с MOEX ---
def fetch_candles(ticker, board='TQBR'):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/{board}/securities/{ticker}/candles.json"
    params = {
        "from": "2000-01-01",
        "interval": 24,
        "iss.meta": "off",
        "iss.json": "extended",
        "candles.columns": "begin,open,high,low,close,value,volume"
    }

    all_data = []
    start = 0

    while True:
        try:
            r = requests.get(url, params={**params, "start": start}, timeout=15)
            r.raise_for_status()
            data = r.json()

            if len(data) < 2 or "candles" not in data[1]:
                break

            candles = data[1]["candles"]
            if not candles:
                break

            all_data.extend(candles)

            start += 500
            sleep(0.2)  # чтобы не получить бан

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            break

    df = pd.DataFrame(all_data, columns=["begin", "open", "high", "low", "close", "value", "volume"])
    df.rename(columns={"begin": "tradedate"}, inplace=True)
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.date  # удаляем время
    df.insert(0, "ticker", ticker)
    df["boardid"] = board

    return df

# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Save to CSV
    df.to_csv(os.path.join(DATA_DIR, f"{ticker}.csv"), index=False)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()

# --- Пример работы ---
if __name__ == "__main__":
    create_tables()
    tickers = tickers_list  # заменишь на свой список

    for ticker in tickers:
        df = fetch_candles(ticker)
        if not df.empty:
            save_data(df)
            print(f"Saved: {ticker} ({len(df)} rows)")
        else:
            print(f"No data for {ticker}")


Saved: ETLN (1290 rows)
Saved: FIXP (1016 rows)
Saved: QIWI (2710 rows)
Saved: OKEY (1073 rows)
Saved: OZON (1088 rows)
Saved: AGRO (2475 rows)


In [20]:
# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Преобразуем tradedate в строку, чтобы избежать ошибок в SQLite
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.strftime('%Y-%m-%d')

    # Append to CSV (if exists)
    csv_path = os.path.join(DATA_DIR, f"{ticker}.csv")
    if os.path.exists(csv_path):
        old_df = pd.read_csv(csv_path, parse_dates=["tradedate"])
        df = pd.concat([old_df, df], ignore_index=True)
        df.drop_duplicates(subset=["ticker", "tradedate"], inplace=True)

    df.to_csv(csv_path, index=False)

    # Преобразуем все данные в строковый формат, чтобы избежать ошибок с типами данных
    df = df.applymap(str)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()



# --- Сбор данных по старому тикеру и сохранение под новым ---
def migrate_ticker(old_ticker, new_ticker):
    df = fetch_candles(old_ticker)
    if df.empty:
        print(f"No data for {old_ticker}")
        return
    df["ticker"] = new_ticker  # переименовываем тикер
    save_data(df)
    print(f"Migrated {old_ticker} → {new_ticker}: {len(df)} rows")

# --- Пример работы ---
if __name__ == "__main__":

   
    # Миграция старых тикеров
    migration_map = {
        "FIVE": "X5",
        "MAIL": "VKCO",
        "CIAN": "CNRU",
        "HHRU": "HEAD"
    }

    for old, new in migration_map.items():
        migrate_ticker(old, new)

Migrated FIVE → X5: 1540 rows
Migrated MAIL → VKCO: 371 rows
Migrated CIAN → CNRU: 803 rows
Migrated HHRU → HEAD: 962 rows


In [7]:
import sqlite3
import pandas as pd

# Подключаемся к базе данных
DB_NAME = "moex_candles.db"
conn = sqlite3.connect(DB_NAME)

# Читаем данные по тикеру YDEX из таблицы candles
query = "SELECT * FROM candles WHERE tradedate = (SELECT max(tradedate) FROM candles)"
df = pd.read_sql(query, conn)

# Закрываем соединение
conn.close()

df


,ticker,tradedate,open,high,low,close,volume,value,boardid
0,CBOM,2025-04-05,8.067,8.087,7.840,8.032,747000.0,5.950738e+06,TQBR
1,ALRS,2025-04-05,48.810,48.970,47.920,48.770,2981700.0,1.439665e+08,TQBR
2,VTBR,2025-04-05,72.020,72.900,72.020,72.320,8499673.0,6.142501e+08,TQBR
3,MDMG,2025-04-05,997.000,1000.000,968.100,993.000,38177.0,3.743930e+07,TQBR
4,RUAL,2025-04-05,34.155,34.455,34.155,34.385,2185010.0,7.475709e+07,TQBR
...,...,...,...,...,...,...,...,...,...
74,ZAYM,2025-04-05,141.400,142.500,138.950,142.500,40440.0,5.656132e+06,TQBR
75,YDEX,2025-04-05,4075.000,4118.000,4006.500,4100.000,135683.0,5.486870e+08,TQBR
76,X5,2025-04-05,3246.500,3313.500,3181.000,3284.500,246557.0,7.989207e+08,TQBR
77,VKCO,2025-04-05,262.300,265.400,260.500,263.100,295576.0,7.740076e+07,TQBR


In [11]:
# 1. Подключаемся к базе
conn = sqlite3.connect("moex_candles.db")

# 2. Получаем список тикеров
tickers = pd.read_sql("SELECT DISTINCT ticker FROM candles", conn)["ticker"].tolist()
tickers

['ABIO',
 'ABRD',
 'AFKS',
 'AFLT',
 'AKRN',
 'ALRS',
 'AMEZ',
 'APRI',
 'APTK',
 'AQUA',
 'ARSA',
 'ASSB',
 'ASTR',
 'AVAN',
 'BANE',
 'BANEP',
 'BELU',
 'BISVP',
 'BLNG',
 'BRZL',
 'BSPB',
 'BSPBP',
 'CARM',
 'CBOM',
 'CHGZ',
 'CHKZ',
 'CHMF',
 'CHMK',
 'CNRU',
 'CNTL',
 'CNTLP',
 'DATA',
 'DELI',
 'DIAS',
 'DIOD',
 'DVEC',
 'DZRD',
 'DZRDP',
 'EELT',
 'ELFV',
 'ELMT',
 'ENPG',
 'ETLN',
 'EUTR',
 'FEES',
 'FESH',
 'FIXP',
 'FLOT',
 'GAZA',
 'GAZAP',
 'GAZC',
 'GAZP',
 'GAZS',
 'GAZT',
 'GCHE',
 'GECO',
 'GEMA',
 'GEMC',
 'GMKN',
 'GTRK',
 'HEAD',
 'HIMCP',
 'HNFG',
 'HYDR',
 'IGST',
 'IGSTP',
 'INGR',
 'IRAO',
 'IRKT',
 'IVAT',
 'JNOS',
 'JNOSP',
 'KAZT',
 'KAZTP',
 'KBSB',
 'KCHE',
 'KCHEP',
 'KGKC',
 'KGKCP',
 'KLSB',
 'KLVZ',
 'KMAZ',
 'KMEZ',
 'KOGK',
 'KRKN',
 'KRKNP',
 'KRKOP',
 'KROT',
 'KROTP',
 'KRSB',
 'KRSBP',
 'KUZB',
 'KZOS',
 'KZOSP',
 'LEAS',
 'LENT',
 'LIFE',
 'LKOH',
 'LMBZ',
 'LNZL',
 'LNZLP',
 'LPSB',
 'LSNG',
 'LSNGP',
 'LSRG',
 'LVHK',
 'MAGE',
 'MAGEP',
 'MAGN',

In [33]:
def clean_specific_tickers():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Список тикеров для обработки
    problem_tickers = ["YDEX", "X5", "VKCO", "CNRU", "HEAD"]
    
    for ticker in problem_tickers:
        print(f"Очистка дубликатов для {ticker}")
        
        # 1. Создаем временную таблицу для данного тикера
        cursor.execute(f"""
            CREATE TEMPORARY TABLE temp_{ticker} AS
            SELECT * FROM candles WHERE ticker = ? AND tradedate NOT LIKE '%00:00:00'
        """, (ticker,))
        
        # 2. Добавляем записи с 00:00:00, только если нет записи с той же датой без времени
        cursor.execute(f"""
            INSERT INTO temp_{ticker} 
            SELECT c.* FROM candles c
            WHERE c.ticker = ? AND c.tradedate LIKE '%00:00:00'
            AND NOT EXISTS (
                SELECT 1 FROM candles c2 
                WHERE c2.ticker = c.ticker 
                AND SUBSTR(c2.tradedate, 1, 10) = SUBSTR(c.tradedate, 1, 10)
                AND c2.tradedate NOT LIKE '%00:00:00'
            )
        """, (ticker,))
        
        # 3. Удаляем оригинальные записи для этого тикера
        cursor.execute("DELETE FROM candles WHERE ticker = ?", (ticker,))
        
        # 4. Вставляем очищенные данные обратно
        cursor.execute(f"""
            INSERT INTO candles 
            SELECT ticker, 
                   SUBSTR(tradedate, 1, 10) AS tradedate, 
                   open, high, low, close, volume, value, boardid 
            FROM temp_{ticker}
        """)
        
        # 5. Удаляем временную таблицу
        cursor.execute(f"DROP TABLE temp_{ticker}")
        
        # 6. Выводим количество обработанных строк
        cursor.execute("SELECT COUNT(*) FROM candles WHERE ticker = ?", (ticker,))
        count = cursor.fetchone()[0]
        print(f"  Обработано {count} строк для {ticker}")
    
    conn.commit()
    conn.close()
    print("Очистка завершена")

In [34]:
if __name__ == "__main__":
    clean_specific_tickers()

Очистка дубликатов для YDEX
  Обработано 2698 строк для YDEX
Очистка дубликатов для X5
  Обработано 1609 строк для X5
Очистка дубликатов для VKCO
  Обработано 1174 строк для VKCO
Очистка дубликатов для CNRU
  Обработано 805 строк для CNRU
Очистка дубликатов для HEAD
  Обработано 1103 строк для HEAD
Очистка завершена


In [5]:
import sqlite3

def ticker_prices(tickers, db_path="moex_candles.db"):
    # Подключение к базе данных
    conn = sqlite3.connect(db_path)

    # Формирование строки запроса
    tickers_placeholder = ", ".join(["?"] * len(tickers))
    query = f"""
        SELECT tradedate, ticker, close
        FROM candles
        WHERE ticker IN ({tickers_placeholder})
        ORDER BY tradedate
    """

    # Выполняем запрос
    df = pd.read_sql(query, conn, params=tickers)
    conn.close()

    # Преобразуем данные в требуемый формат
    df_pivot = df.pivot(index="tradedate", columns="ticker", values="close")
    df_pivot.index = pd.to_datetime(df_pivot.index)  # Преобразуем TRADEDATE в формат даты
    df_pivot = df_pivot.sort_index()  # Сортируем по дате
    
    return df_pivot


In [6]:
tickers = ["SBER", "LKOH"]  # Пример списка тикеров
result = ticker_prices(tickers=tickers)
print(result)

ticker        LKOH    SBER
tradedate                 
2003-08-20   581.0     NaN
2003-08-21   589.5     NaN
2003-08-22   589.0     NaN
2003-08-25   586.5     NaN
2003-08-26   581.0     NaN
...            ...     ...
2025-04-01  6905.0  303.24
2025-04-02  6978.0  306.11
2025-04-03  6802.0  302.09
2025-04-04  6445.5  285.35
2025-04-05  6486.0  284.98

[5415 rows x 2 columns]


In [3]:
from optimizer import optimizer_for_tickers

tickers = ['SBER', 'LKOH']
print(optimizer_for_tickers(tickers, rf=0.02))

Expected annual return: 11.9%
Annual volatility: 35.0%
Sharpe Ratio: 0.28
{'weights': {'LKOH': 1.0, 'SBER': 0.0}, 'performance': {'return': np.float64(0.11884758235818213), 'volatility': np.float64(0.34990123514651006), 'sharpe_ratio': np.float64(0.28250138161642335)}}


In [4]:
from get_prices_sql import get_tickers_prices_sql

get_tickers_prices_sql(tickers)

ticker,LKOH,SBER
tradedate,,
2003-08-20,581.0,NaN
2003-08-21,589.5,NaN
2003-08-22,589.0,NaN
2003-08-25,586.5,NaN
2003-08-26,581.0,NaN
...,...,...
2025-04-01,6905.0,303.24
2025-04-02,6978.0,306.11
2025-04-03,6802.0,302.09


In [37]:
conn = sqlite3.connect("moex_candles.db")
tickers = pd.read_sql("SELECT * FROM candles WHERE ticker = 'SBER'", conn)
tickers

,ticker,tradedate,open,high,low,close,volume,value,boardid
0,SBER,2007-07-20,109.00,111.11,108.01,109.20,20252489.0,2.211623e+09,TQBR
1,SBER,2007-07-23,109.70,112.65,108.10,112.00,35092029.0,3.901829e+09,TQBR
2,SBER,2007-07-24,112.10,113.05,109.47,109.53,31762001.0,3.536964e+09,TQBR
3,SBER,2007-07-25,108.50,109.98,107.10,109.63,39700937.0,4.300853e+09,TQBR
4,SBER,2007-07-26,110.00,110.42,103.30,104.18,49871205.0,5.299123e+09,TQBR
...,...,...,...,...,...,...,...,...,...
4439,SBER,2025-04-02,303.23,306.50,301.21,306.11,40207020.0,1.223946e+10,TQBR
4440,SBER,2025-04-03,306.21,309.20,296.00,302.09,52408560.0,1.583042e+10,TQBR
4441,SBER,2025-04-04,303.55,304.37,284.40,285.35,88719200.0,2.603664e+10,TQBR
4442,SBER,2025-04-05,285.25,287.03,280.11,284.98,14100160.0,3.989186e+09,TQBR


In [12]:
import pandas as pd
import sqlite3
from data_loader import get_ticker_history
from datetime import datetime, timedelta

In [33]:
def delete_last_row(ticker, db_path="moex_candles.db", table_name="candles"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(f"""
        DELETE FROM {table_name}
        WHERE ticker = ?
          AND tradedate = (
              SELECT MAX(tradedate)
              FROM {table_name}
              WHERE ticker = ?
          );
    """, (ticker, ticker))

    conn.commit()
    print(f"Удалена последняя строка для тикера {ticker}")
    conn.close()

In [34]:
delete_last_row('SBER')

Удалена последняя строка для тикера SBER


In [35]:
# 1. Подключаемся к базе
conn = sqlite3.connect("moex_candles.db")
ticker = 'SBER'

query = f"SELECT MAX(tradedate) as last_date FROM candles WHERE ticker = '{ticker}'"
last_date = pd.read_sql(query, conn)["last_date"].iloc[0]
last_date = datetime.strptime(last_date, "%Y-%m-%d").date()

# 4. Скачиваем новые данные, начиная со следующего дня
start_date = last_date + timedelta(days=1)
end_date = datetime.today().date()

if start_date < end_date:
    # тут ты подгружаешь данные с MOEX или из файла/парсера
    df_new = get_ticker_history(ticker, start_date, end_date)

conn.close()

df_new

,TRADEDATE,SBER
0,2025-04-06,290.35
